In [23]:
import sys
sys.path.insert(0, '../')

import pandas as pd
from sklearn import metrics
#from keras.wrappers.scikit_learn import KerasClassifier
from classification_plots import plot_confusion_matrix
import matplotlib.pyplot as plt
import os
from collections import Counter
import numpy as np
import scipy 
%matplotlib notebook
import shutil
from scipy.io import loadmat

In [24]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import warnings
warnings.filterwarnings('ignore')
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [25]:
audio_dir = '../../ESC-50/audio/'
path_to_db='../../ESC-50/'

In [4]:
import time
for fileList in os.walk(audio_dir):
    count = 0
    for fname in fileList[2]:
        t1 = time.time()
        count += 1
        name_array = fname.split('.')
        name = name_array[0]
        name_array = name_array[0].split('-')
        curr_class = name_array[-1]
        if(name != '' and curr_class != ''):
            fpath = audio_dir + fname


            save_dir = './class_audio/class' + str(curr_class) + '/'
            
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            
            shutil.copy(fpath,save_dir + name + '.wav')
        t2 = time.time()
        print(count, t2-t1)
    

1 0.0958702564239502
2 0.024005413055419922
3 0.027925729751586914
4 0.04209089279174805
5 0.0199735164642334
6 0.015978336334228516
7 0.0199737548828125
8 0.05596160888671875
9 0.06391263008117676
10 0.012672662734985352
11 0.027965307235717773
12 0.01993870735168457
13 0.043976545333862305
14 0.043908119201660156
15 0.023964881896972656
16 0.016013145446777344
17 0.023966073989868164
18 0.0199739933013916
19 0.012024164199829102
20 0.03191709518432617
21 0.020645618438720703
22 0.027881145477294922
23 0.04394030570983887
24 0.0199735164642334
25 0.055959224700927734
26 0.027927398681640625
27 0.01601266860961914
28 0.020557641983032227
29 0.019974231719970703
30 0.015978574752807617
31 0.03595161437988281
32 0.015979766845703125
33 0.01997232437133789
34 0.027962684631347656
35 0.015978574752807617
36 0.03994560241699219
37 0.03595161437988281
38 0.01597881317138672
39 0.015984773635864258
40 0.04793524742126465
41 0.019975662231445312
42 0.019974708557128906
43 0.023966073989868164


348 0.01993536949157715
349 0.051962852478027344
350 0.01594376564025879
351 0.02000713348388672
352 0.01742410659790039
353 0.01601696014404297
354 0.051892757415771484
355 0.039946556091308594
356 0.01997208595275879
357 0.01597881317138672
358 0.019974708557128906
359 0.05094170570373535
360 0.027962207794189453
361 0.015979290008544922
362 0.015978574752807617
363 0.04095578193664551
364 0.01597881317138672
365 0.023969173431396484
366 0.02396678924560547
367 0.019488096237182617
368 0.21640324592590332
369 0.0953516960144043
370 0.023969411849975586
371 0.04793405532836914
372 0.016566753387451172
373 0.015975236892700195
374 0.019977092742919922
375 0.0159757137298584
376 0.027961015701293945
377 0.011984109878540039
378 0.03147411346435547
379 0.023966312408447266
380 0.019973278045654297
381 0.016483545303344727
382 0.015962600708007812
383 0.0199735164642334
384 0.027962207794189453
385 0.019973278045654297
386 0.015979528427124023
387 0.019973278045654297
388 0.01598024368286

689 0.03195691108703613
690 0.011745691299438477
691 0.0199737548828125
692 0.011983394622802734
693 0.0199739933013916
694 0.01598048210144043
695 0.015976667404174805
696 0.027986526489257812
697 0.01595449447631836
698 0.019974231719970703
699 0.015979528427124023
700 0.015976428985595703
701 0.0163266658782959
702 0.019973278045654297
703 0.011984109878540039
704 0.0199735164642334
705 0.015979528427124023
706 0.01198267936706543
707 0.017529726028442383
708 0.015937089920043945
709 0.015978336334228516
710 0.01597881317138672
711 0.019974946975708008
712 0.026513099670410156
713 0.015941143035888672
714 0.019973278045654297
715 0.011986017227172852
716 0.015978574752807617
717 0.016024351119995117
718 0.015932321548461914
719 0.017583370208740234
720 0.015975475311279297
721 0.015980005264282227
722 0.020008325576782227
723 0.01594376564025879
724 0.015977859497070312
725 0.01597905158996582
726 0.015979290008544922
727 0.02001023292541504
728 0.01597762107849121
729 0.01594233512

1026 0.0199735164642334
1027 0.011984109878540039
1028 0.015979528427124023
1029 0.015978097915649414
1030 0.015979528427124023
1031 0.01597738265991211
1032 0.031957387924194336
1033 0.02796339988708496
1034 0.3118896484375
1035 0.04836845397949219
1036 0.01597905158996582
1037 0.01597738265991211
1038 0.01597905158996582
1039 0.015978574752807617
1040 0.01789259910583496
1041 0.0199737548828125
1042 0.015977144241333008
1043 0.015979290008544922
1044 0.01597881317138672
1045 0.018491268157958984
1046 0.023968219757080078
1047 0.019979000091552734
1048 0.015974998474121094
1049 0.0159757137298584
1050 0.015978336334228516
1051 0.019957542419433594
1052 0.03195691108703613
1053 0.01597881317138672
1054 0.01598215103149414
1055 0.020022153854370117
1056 0.011984109878540039
1057 0.0199737548828125
1058 0.015978574752807617
1059 0.015978574752807617
1060 0.019972801208496094
1061 0.06791114807128906
1062 0.14020442962646484
1063 0.011983394622802734
1064 0.015978574752807617
1065 0.01599

1348 0.26374125480651855
1349 0.05995941162109375
1350 0.015952110290527344
1351 0.01597905158996582
1352 0.015978336334228516
1353 0.01597881317138672
1354 0.019973278045654297
1355 0.015980243682861328
1356 0.016013383865356445
1357 0.015943050384521484
1358 0.015978336334228516
1359 0.012156009674072266
1360 0.01997208595275879
1361 0.019974231719970703
1362 0.011983871459960938
1363 0.019973278045654297
1364 0.016015291213989258
1365 0.016269922256469727
1366 0.02792501449584961
1367 0.0199737548828125
1368 0.011985301971435547
1369 0.027961015701293945
1370 0.019006013870239258
1371 0.015979528427124023
1372 0.015978097915649414
1373 0.020007848739624023
1374 0.015944719314575195
1375 0.017256736755371094
1376 0.015979766845703125
1377 0.015978574752807617
1378 0.01597905158996582
1379 0.019977807998657227
1380 0.01597309112548828
1381 0.016020536422729492
1382 0.016015291213989258
1383 0.019971132278442383
1384 0.011949300765991211
1385 0.02000737190246582
1386 0.0159466266632080

1674 0.03998303413391113
1675 0.031920671463012695
1676 0.015980005264282227
1677 0.016399383544921875
1678 0.016012191772460938
1679 0.015945911407470703
1680 0.039947509765625
1681 0.037140846252441406
1682 0.0199735164642334
1683 0.015978336334228516
1684 0.015978574752807617
1685 0.015977859497070312
1686 0.015878915786743164
1687 0.015978336334228516
1688 0.019974231719970703
1689 0.011982440948486328
1690 0.015979766845703125
1691 0.011984109878540039
1692 0.028585433959960938
1693 0.01997518539428711
1694 0.015978097915649414
1695 0.019974231719970703
1696 0.04464268684387207
1697 0.039946556091308594
1698 0.04801583290100098
1699 0.019932985305786133
1700 0.016015052795410156
1701 0.14780664443969727
1702 0.01960468292236328
1703 0.03998374938964844
1704 0.27182626724243164
1705 0.04793572425842285
1706 0.020006895065307617
1707 0.011950016021728516
1708 0.022483110427856445
1709 0.035950660705566406
1710 0.049488067626953125
1711 0.01597905158996582
1712 0.011983633041381836
1

1997 0.016016483306884766
1998 0.015942096710205078
1999 0.016013622283935547
2000 0.027962207794189453


In [26]:
from torch.utils.data import Dataset, DataLoader
class audiotrainset(Dataset):
    def __init__(self):
        root = "./class_audio/"
        self.data_list = []
        self.label_list = []
        for root, dir, files in os.walk("./class_audio/"):
            for file in files:
                if file.find('.mat')!= -1:
#                     print(os.path.join(root, file))
#                     print(int(root[13:]))
                    mdict = loadmat(os.path.join(root, file))
                    data = mdict['file_features']
                    labels = mdict['label']
                    self.data_list.append(data)
                    self.label_list.append(labels)
        print(len(self.data_list), len(self.label_list))
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):

        data = self.data_list[index]
        label= self.label_list[index] 
        data = np.asarray(data)
        label = np.asarray(label[0][0])
        data = data.reshape((data.shape[1],))
        return data, label
audiodataloader = audiotrainset()
data,label = audiodataloader.__getitem__(1361)
print(label)

2000 2000
4


In [5]:
## Extracts LPC coefficients ###
import numpy as np
import scipy as sp
import scipy.signal as sig

def lpc_ref(signal, order):
    """Compute the Linear Prediction Coefficients.
    Return the order + 1 LPC coefficients for the signal. c = lpc(x, k) will
    find the k+1 coefficients of a k order linear filter:
      xp[n] = -c[1] * x[n-2] - ... - c[k-1] * x[n-k-1]
    Such as the sum of the squared-error e[i] = xp[i] - x[i] is minimized.
    Parameters
    ----------
    signal: array_like
        input signal
    order : int
        LPC order (the output will have order + 1 items)
    Note
    ----
    This is just for reference, as it is using the direct inversion of the
    toeplitz matrix, which is really slow"""
    if signal.ndim > 1:
        raise ValueError("Array of rank > 1 not supported yet")
#     if order > signal.size:
#         raise ValueError("Input signal must have a lenght >= lpc order")

    if order > 0:
        p = order + 1
        r = np.zeros(p, signal.dtype)
        # Number of non zero values in autocorrelation one needs for p LPC
        # coefficients
        nx = np.min([p, signal.size])
        x = np.correlate(signal, signal, 'full')
        r[:nx] = x[signal.size-1:signal.size+order]
        phi = np.dot(sp.linalg.inv(sp.linalg.toeplitz(r[:-1])), -r[1:])
        return np.concatenate(([1.], phi))
    else:
        return np.ones(1, dtype = signal.dtype)

def levinson_1d(r, order):
    """Levinson-Durbin recursion, to efficiently solve symmetric linear systems
    with toeplitz structure.
    Arguments
    ---------
        r : array-like
            input array to invert (since the matrix is symmetric Toeplitz, the
            corresponding pxp matrix is defined by p items only). Generally the
            autocorrelation of the signal for linear prediction coefficients
            estimation. The first item must be a non zero real.
    Note
    ----
    This implementation is in python, hence unsuitable for any serious
    computation. Use it as educational and reference purpose only.
    Levinson is a well-known algorithm to solve the Hermitian toeplitz
    equation:
                       _          _
        -R[1] = R[0]   R[1]   ... R[p-1]    a[1]
         :      :      :          :      *  :
         :      :      :          _      *  :
        -R[p] = R[p-1] R[p-2] ... R[0]      a[p]
                       _
    with respect to a (  is the complex conjugate). Using the special symmetry
    in the matrix, the inversion can be done in O(p^2) instead of O(p^3).  
    """
    r = np.atleast_1d(r)
    if r.ndim > 1:
        raise ValueError("Only rank 1 are supported for now.")

    n = r.size
    if order > n - 1:
        raise ValueError("Order should be <= size-1")
    elif n < 1:
        raise ValueError("Cannot operate on empty array !")

    if not np.isreal(r[0]):
        raise ValueError("First item of input must be real.")
    elif not np.isfinite(1/r[0]):
        raise ValueError("First item should be != 0")

    # Estimated coefficients
    a = np.empty(order+1, r.dtype)
    # temporary array
    t = np.empty(order+1, r.dtype)
    # Reflection coefficients
    k = np.empty(order, r.dtype)

    a[0] = 1.
    e = r[0]

    for i in xrange(1, order+1):
        acc = r[i]
        for j in range(1, i):
            acc += a[j] * r[i-j]
        k[i-1] = -acc / e
        a[i] = k[i-1]

        for j in range(order):
            t[j] = a[j]

        for j in range(1, i):
            a[j] += k[i-1] * np.conj(t[i-j])

        e *= 1 - k[i-1] * np.conj(k[i-1])

    return a, e, k

In [6]:
######## Wavelet Decomposition #######
import pywt
def wavelet_decompose(y,wtype):
    wt = pywt.Wavelet(wtype)
    cA = pywt.wavedec(y,wt,level=2)[0]
    cA = np.asarray(cA)
    return cA

In [7]:
##### Stats from a signal #####
def get_stats(y):
    split_array = np.array_split(y,4)
    stats_vector = np.zeros(len(split_array)*2)
    offset = 0
    for idx,chunk in enumerate(split_array,0):
        stats_vector[offset] = np.mean(chunk)
        stats_vector[offset + 1] = np.var(chunk)
        offset = offset + 2
    stats_vector = np.asarray(stats_vector)
    return (stats_vector)
    

In [8]:
######### Extract MFCC Coefficients #########
def extract_mfcc(y,sr):
    mfcc_array = librosa.feature.mfcc(y,sr,n_mfcc=13)
    mfcc_features = []
    for i in range(13):
        mfcc_wav = wavelet_decompose(mfcc_array[i,:],'haar')
        lpc_coeffs = lpc_ref(mfcc_wav,5)[1:]
        mfcc_features.append(np.asarray(lpc_coeffs))
    
    
    final_coeffs = 0
    for coeff in mfcc_features:
        final_coeffs = np.concatenate((final_coeffs,coeff),axis = None)
    
    final_coeffs = final_coeffs[1:]
    
    final_coeffs = np.asarray(final_coeffs) 
    return (final_coeffs)

In [9]:
def feature_extraction(y):
    sr = 44100
    f1 = (lpc_ref(y,9)[1:])
    f2 = (get_stats(wavelet_decompose(y,'haar')))
    f3 = (extract_mfcc(y,sr))
    f4 = (get_stats(wavelet_decompose(y,'db4')))
    f5 = (get_stats(wavelet_decompose(y,'sym4')))
    local_features =  np.concatenate((f1,f2,f3,f4,f5),axis = None)
    return local_features

In [27]:
import torch.nn as nn
import torch.nn.functional as F
class SpectralNet(nn.Module):
    def __init__(self,):
        super(SpectralNet, self).__init__()
        self.fc1 = nn.Linear(87,40)
        self.fc2 = nn.Linear(40,40)
        self.fc3 = nn.Linear(40,40)
        self.fc4 = nn.Linear(40,50)
        self.softmax = nn.Softmax()
        self.batchnorm = nn.BatchNorm1d(40)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.batchnorm(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.batchnorm(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = self.softmax(x)
        return x

model = SpectralNet()
print (model)

SpectralNet(
  (fc1): Linear(in_features=87, out_features=40, bias=True)
  (fc2): Linear(in_features=40, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=50, bias=True)
  (softmax): Softmax(dim=None)
  (batchnorm): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [11]:
import time
for fileList in os.walk(audio_dir):
    count = 0
    for fname in fileList[2]:
        t1 = time.time()
        count += 1
        name_array = fname.split('.')
        name = name_array[0]
        name_array = name_array[0].split('-')
        curr_class = name_array[-1]
        if(name != '' and curr_class != ''):
            fpath = audio_dir + fname
            (sig, rate) = librosa.load(fpath, sr=None, mono=True,  dtype=np.float32)
            features = feature_extraction(sig)

            save_dir = './data3/class' + str(curr_class) + '/'
            
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            np.save(save_dir + name + '.npy',features)
        t2 = time.time()
        print(count, t2-t1)

1 4.958807468414307
2 4.393475294113159
3 4.422207832336426
4 4.7833685874938965
5 6.105371475219727
6 7.383301496505737
7 6.553909778594971
8 4.9680445194244385
9 4.748013973236084
10 4.4298996925354
11 5.005567789077759
12 5.0864527225494385
13 6.49432110786438
14 4.9658896923065186
15 6.446439743041992
16 5.449322700500488
17 7.093367099761963
18 6.592381954193115
19 5.182332515716553
20 4.770303010940552
21 4.312694072723389
22 7.797826766967773
23 9.735795021057129
24 9.104111433029175
25 8.853600025177002
26 9.903822183609009
27 9.423120737075806
28 8.353999376296997
29 9.10326886177063
30 7.922693729400635
31 8.711971044540405
32 9.486010074615479
33 10.029250144958496
34 9.100651025772095
35 9.17475700378418
36 9.265475273132324
37 9.03073239326477
38 9.39542555809021
39 9.21373438835144
40 10.418880701065063
41 9.825042486190796
42 8.936200380325317
43 8.94554877281189
44 9.311920166015625
45 9.260784387588501
46 8.856303691864014
47 9.979761362075806
48 9.126184225082397
49 8

736 3.463151454925537
737 3.217008113861084
738 3.4479215145111084
739 3.0893468856811523
740 3.242131471633911
741 3.296806573867798
742 3.061863660812378
743 3.5052475929260254
744 3.0665781497955322
745 3.554464340209961
746 3.047795057296753
747 3.556133508682251
748 2.964679479598999
749 3.517219305038452
750 3.0549588203430176
751 3.4408931732177734
752 3.1090872287750244
753 2.9688804149627686
754 3.5122315883636475
755 3.0389294624328613
756 3.5780389308929443
757 3.052750587463379
758 3.5649492740631104
759 3.043482780456543
760 3.570589303970337
761 3.050337076187134
762 3.559098482131958
763 3.415006160736084
764 3.351658344268799
765 3.0659420490264893
766 3.4753315448760986
767 3.0629115104675293
768 3.5630791187286377
769 3.058995485305786
770 3.5720763206481934
771 3.049955368041992
772 3.5511507987976074
773 3.0745348930358887
774 3.5872621536254883
775 3.0374319553375244
776 3.5685982704162598
777 2.9761550426483154
778 3.3886425495147705
779 3.1459758281707764
780 3.2

1444 3.104172468185425
1445 3.5052454471588135
1446 3.0669422149658203
1447 3.5750391483306885
1448 3.16304087638855
1449 3.4985573291778564
1450 3.120903491973877
1451 3.5210254192352295
1452 3.1845996379852295
1453 3.4912045001983643
1454 3.1217517852783203
1455 3.512230157852173
1456 3.1159725189208984
1457 3.4851009845733643
1458 3.095078468322754
1459 3.5681540966033936
1460 3.0846288204193115
1461 3.7198288440704346
1462 3.3075475692749023
1463 3.302649974822998
1464 3.3261821269989014
1465 3.2888269424438477
1466 3.0846292972564697
1467 3.4773313999176025
1468 3.445430278778076
1469 3.3227362632751465
1470 3.2102720737457275
1471 3.4030401706695557
1472 3.2049760818481445
1473 3.419971227645874
1474 3.0713562965393066
1475 3.5045504570007324
1476 3.073974132537842
1477 3.5641133785247803
1478 3.0918619632720947
1479 3.5541059970855713
1480 3.1287620067596436
1481 3.520112991333008
1482 3.0788886547088623
1483 3.5080668926239014
1484 3.083068609237671
1485 3.585042715072632
1486 

In [33]:
import torch.optim as optim
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import time

device = 0
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.7)

validation_split = .25
random_seed= 42
shuffle_dataset = True
dataset_size = 2000
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

dataset  = audiotrainset()
trainloader = torch.utils.data.DataLoader(dataset, batch_size=128, 
                                           sampler=train_sampler,pin_memory=True)
valloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                                sampler=valid_sampler,pin_memory=True)

val_history = []
val_loss_hist = []
train_history = []
train_loss_hist = []

for epoch in range(300):  # loop over the dataset multiple times
    
    count = 0
    running_accuracy = 0.0
    running_loss = 0.0
    t1 = time.time()
    for i, data in enumerate(valloader, 0):
        count += 1
        inputs, labels = data
        inputs = inputs.float()#.to(device)
        labels = labels.long()#.to(device)
        outputs = model(inputs)
        val_loss = criterion(outputs, labels)
        print(val_loss.item())
        _, preds = torch.max(outputs, 1)
        acc_val = torch.eq(preds, labels).float().mean()
        running_accuracy = running_accuracy + acc_val.item()
        running_loss = running_loss + val_loss.item()
        
        
    running_accuracy = running_accuracy/count
    running_loss = running_loss/count
    val_history.append(running_accuracy)
    val_loss_hist.append(running_loss)
    t2 = time.time()
    print("===========Phase: Val============")
    print("Validation Time: {}".format(t2 - t1))
    print("Epoch: {}  val_loss: {}".format(epoch, running_loss))
    print("Epoch: {}  val_accuracy: {}".format(epoch, running_accuracy))
    
    
    #### Training #####
    running_loss = 0.0
    count = 0
    running_accuracy = 0
    t1 = time.time()
    for i, data in enumerate(trainloader, 0):
        count += 1
        inputs, labels = data
        inputs = inputs.float()
        inputs = inputs.float()#.to(device)
        labels = labels.long()#.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        running_loss = running_loss + loss.item()
        _, preds = torch.max(outputs, 1)
        acc_train = torch.eq(preds, labels).float().mean()
        running_accuracy = running_accuracy + acc_train.item()
        
    
    running_accuracy = running_accuracy/count
    running_loss = running_loss/count
    train_history.append(running_accuracy)
    train_loss_hist.append(running_loss)
    t2 = time.time()
    print("===========Phase: Train============") 
    print("Training Time: {}".format(t2 - t1))
    print("Epoch: {}  train_loss: {}".format(epoch, running_loss))
    print("Epoch: {}  train_accuracy: {}".format(epoch, running_accuracy))
    print()
        
print('Finished Training')


2000 2000
nan
nan
nan
nan
===========Phase: Val============
Validation Time: 0.21450424194335938
Epoch: 0  val_loss: nan
Epoch: 0  val_accuracy: 0.00996767240576446
===========Phase: Train============
Training Time: 0.0747988224029541
Epoch: 0  train_loss: nan
Epoch: 0  train_accuracy: 0.022786458333333332

nan
nan
nan
nan
===========Phase: Val============
Validation Time: 0.035903215408325195
Epoch: 1  val_loss: nan
Epoch: 1  val_accuracy: 0.010169719811528921
===========Phase: Train============
Training Time: 0.0718073844909668
Epoch: 1  train_loss: nan
Epoch: 1  train_accuracy: 0.02304121378498773

nan
nan
nan
nan
===========Phase: Val============
Validation Time: 0.03390836715698242
Epoch: 2  val_loss: nan
Epoch: 2  val_accuracy: 0.00996767240576446
===========Phase: Train============
Training Time: 0.07080984115600586
Epoch: 2  train_loss: nan
Epoch: 2  train_accuracy: 0.023295969236642122

nan
nan
nan
nan
===========Phase: Val============
Validation Time: 0.03390860557556152
Epoc

KeyboardInterrupt: 